# BRSR Disability Data Extraction - All NSE Companies

**Extracts disability employment data from 1,233 NSE-listed companies**

- Runtime: ~1 hour
- Rate limited: 2 seconds per company
- Auto-saves checkpoints every 50 companies

## Step 1: Import Libraries

In [1]:
import json
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import time
import os
from IPython.display import clear_output

print('✅ All libraries imported successfully')

✅ All libraries imported successfully


## Step 2: Setup Folders & Configuration

In [2]:
# Create output folders
os.makedirs('firm_data_xml', exist_ok=True)
os.makedirs('firm_data_csv', exist_ok=True)
os.makedirs('brsr_results', exist_ok=True)

# Configuration
RATE_LIMIT_DELAY = 2  # Seconds between requests (DON'T go lower!)
SAVE_PROGRESS_EVERY = 50  # Save checkpoint every N companies
MAX_RETRIES = 3  # Retry failed downloads

print('✅ Folders created')
print(f'⏱️  Rate limit: {RATE_LIMIT_DELAY} seconds per company')
print(f'💾 Checkpoints every {SAVE_PROGRESS_EVERY} companies')
print(f'🔄 Max retries per company: {MAX_RETRIES}')

✅ Folders created
⏱️  Rate limit: 2 seconds per company
💾 Checkpoints every 50 companies
🔄 Max retries per company: 3


## Step 3: Load Company List from GitHub

**Loading directly from GitHub - no upload needed!**

In [3]:
# Load the NSE company list from GitHub
csv_url = 'https://raw.githubusercontent.com/c-malhotra/c-malhotra.github.io/refs/heads/main/PC5/all_firm_links.csv'
all_firms_df = pd.read_csv(csv_url)

# Clean column names (removes newlines, **, and spaces)
all_firms_df.columns = all_firms_df.columns.str.strip().str.replace('\n', '').str.replace('**', '')

print(f'✅ Loaded {len(all_firms_df)} companies from GitHub')
print(f'📋 Columns: {list(all_firms_df.columns)}')
print('\nFirst 3 companies:')
all_firms_df.head(3)

✅ Loaded 1227 companies from GitHub
📋 Columns: ['COMPANY', 'FROM YEAR', 'TO YEAR', 'ATTACHMENT', 'XBRL', 'ORIGINAL SUBMISSION DATE', 'LATEST REVISION DATE']

First 3 companies:


,COMPANY,FROM YEAR,TO YEAR,ATTACHMENT,XBRL,ORIGINAL SUBMISSION DATE,LATEST REVISION DATE
0,RAJESH EXPORTS LIMITED,2024,2025,https://nsearchives.nseindia.com/corporate/RAJ...,https://nsearchives.nseindia.com/corporate/xbr...,19-Dec-2025,-
1,BF UTILITIES LIMITED,2024,2025,https://nsearchives.nseindia.com/corporate/BFU...,https://nsearchives.nseindia.com/corporate/xbr...,27-Nov-2025,-
2,Network People Services Technologies Limited,2024,2025,https://nsearchives.nseindia.com/corporate/NPS...,https://nsearchives.nseindia.com/corporate/xbr...,24-Nov-2025,-


## Step 4: Main Extraction Loop

**This will process ALL 1,233 companies**  
**Expected runtime: ~60 minutes**

Progress updates every 10 companies.

In [ ]:
# Initialize tracking
results = []
failed_companies = []
total_companies = len(all_firms_df)

print(f'🚀 Starting extraction of {total_companies} companies...')
print('=' * 70)

# MAIN LOOP - Process each company
for idx, row in all_firms_df.iterrows():

    company_name = row['COMPANY']
    xml_url = row['XBRL']  # Changed from 'xml_urls'

    # Progress display (every 10 companies)
    if idx % 10 == 0:
        clear_output(wait=True)
        print(f'📊 Progress: {idx}/{total_companies} ({idx/total_companies*100:.1f}%)')
        print(f'✅ Successful: {len(results)} | ⚠️  Failed: {len(failed_companies)}')
        if results:
            df_temp = pd.DataFrame(results)
            avg_pct = df_temp['disabled_percent'].mean()
            print(f'📈 Avg disabled employment so far: {avg_pct:.2f}%')
        print(f'\nCurrently processing: {company_name}')

    # Skip if no XML URL
    if pd.isna(xml_url) or xml_url == '':
        failed_companies.append({'company': company_name, 'reason': 'No XML URL'})
        continue

    xml_filename = xml_url.split('/')[-1]
    xml_path = f'firm_data_xml/{xml_filename}'

    # ==================== DOWNLOAD XML ====================
    download_success = False

    # Check if already downloaded
    if os.path.exists(xml_path):
        download_success = True
    else:
        # Try downloading with retries
        for attempt in range(MAX_RETRIES):
            try:
                response = requests.get(
                    xml_url,
                    headers={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'},
                    timeout=30
                )

                if response.status_code == 200:
                    with open(xml_path, 'wb') as f:
                        f.write(response.content)
                    download_success = True
                    time.sleep(RATE_LIMIT_DELAY)  # Rate limiting!
                    break

            except Exception as e:
                if attempt == MAX_RETRIES - 1:
                    failed_companies.append({'company': company_name, 'reason': f'Download error: {e}'})
                time.sleep(3)

    if not download_success:
        continue

    # ==================== PARSE XML ====================
    try:
        tree = ET.parse(xml_path)
        root = tree.getroot()

        # Structural elements to skip
        skip_tags = {
            'xbrl', 'context', 'entity', 'identifier', 'period', 'startDate',
            'endDate', 'instant', 'scenario', 'typedMember', 'unit', 'measure',
            'schemaRef', 'segment'
        }

        rows = []
        for elem in root.iter():
            tag = elem.tag.split('}')[-1] if '}' in elem.tag else elem.tag

            if tag in skip_tags:
                continue

            if 'Domain' in tag and elem.text and 'Domain' in elem.text:
                continue

            ctx = elem.attrib.get('contextRef', '')

            if ctx and elem.text is not None:
                rows.append({
                    'Element Name': tag,
                    'Period/Context': ctx,
                    'Value': elem.text.strip()
                })

        temp_df = pd.DataFrame(rows)

        # Save parsed data
        csv_filename = xml_filename.split('.')[0] + '.csv'
        temp_df.to_csv(f'firm_data_csv/{csv_filename}', index=False)

        # ==================== EXTRACT DISABILITY DATA ====================
        # Total employees
        total_emp_row = temp_df[
            (temp_df['Element Name'] == 'NumberOfEmployeesOrWorkersIncludingDifferentlyAbled') &
            (temp_df['Period/Context'] == 'D_Gender_Employees_TableA')
        ]
        total_employees = int(total_emp_row['Value'].iloc[0]) if not total_emp_row.empty else None

        # Disabled employees
        disabled_emp_row = temp_df[
            (temp_df['Element Name'] == 'NumberOfDifferentlyAbledEmployeesOrWorkers') &
            (temp_df['Period/Context'] == 'D_Gender_Employees_TableB')
        ]
        disabled_employees = int(disabled_emp_row['Value'].iloc[0]) if not disabled_emp_row.empty else None

        # Calculate percentage
        if total_employees and disabled_employees is not None:
            disabled_percent = (disabled_employees / total_employees) * 100
        else:
            disabled_percent = None

        results.append({
            'company': company_name,
            'total_employees': total_employees,
            'disabled_employees': disabled_employees,
            'disabled_percent': disabled_percent,
            'year': row['TO YEAR'],
            'xml_url': xml_url
        })

    except Exception as e:
        failed_companies.append({'company': company_name, 'reason': f'Parsing error: {str(e)}'})

    # ==================== SAVE CHECKPOINTS ====================
    if (idx + 1) % SAVE_PROGRESS_EVERY == 0:
        df_checkpoint = pd.DataFrame(results)
        df_checkpoint.to_csv(f'brsr_results/checkpoint_{idx+1}.csv', index=False)
        print(f'\n💾 Checkpoint saved at company {idx+1}')

print('\n' + '=' * 70)
print('🎉 EXTRACTION COMPLETE!')
print('=' * 70)

🚀 Starting extraction of 1227 companies...
📊 Progress: 0/1227 (0.0%)
✅ Successful: 0 | ⚠️  Failed: 0

Currently processing: RAJESH EXPORTS LIMITED
📊 Progress: 10/1227 (0.8%)
✅ Successful: 10 | ⚠️  Failed: 0
📈 Avg disabled employment so far: 1.26%

Currently processing: IFCI Limited
📊 Progress: 20/1227 (1.6%)
✅ Successful: 20 | ⚠️  Failed: 0
📈 Avg disabled employment so far: 0.97%

Currently processing: Nagarjuna Fertilizers and Chemicals Limited
📊 Progress: 30/1227 (2.4%)
✅ Successful: 30 | ⚠️  Failed: 0
📈 Avg disabled employment so far: 0.71%

Currently processing: PC Jeweller Limited
📊 Progress: 40/1227 (3.3%)
✅ Successful: 40 | ⚠️  Failed: 0
📈 Avg disabled employment so far: 0.56%

Currently processing: SPACENET ENTERPRISES INDIA LIMITED

💾 Checkpoint saved at company 50
📊 Progress: 50/1227 (4.1%)
✅ Successful: 50 | ⚠️  Failed: 0
📈 Avg disabled employment so far: 0.45%

Currently processing: VERANDA LEARNING SOLUTIONS LIMITED
📊 Progress: 60/1227 (4.9%)
✅ Successful: 60 | ⚠️  Failed:

## Step 5: Save Final Results

In [6]:
# Save final combined results
df_final = pd.DataFrame(results)
df_final.to_csv('brsr_results/all_companies_disability_data.csv', index=False)
print(f'✅ Final results saved: {len(results)} companies')

# Save failed companies log
if failed_companies:
    df_failed = pd.DataFrame(failed_companies)
    df_failed.to_csv('brsr_results/failed_companies.csv', index=False)
    print(f'⚠️  Failed companies logged: {len(failed_companies)}')

print('\n✅ All files saved to brsr_results/ folder')

✅ Final results saved: 1227 companies

✅ All files saved to brsr_results/ folder


## Step 6: View Results & Statistics

In [7]:
# Load and analyze results
df_results = pd.read_csv('brsr_results/all_companies_disability_data.csv')

# Clean data (remove None/NaN)
df_clean = df_results[df_results['total_employees'].notna() & df_results['disabled_employees'].notna()].copy()

print('📊 SUMMARY STATISTICS')
print('=' * 70)
print(f'Total companies processed: {len(df_results)}')
print(f'Companies with valid data: {len(df_clean)}')
print(f'\nAggregate Employment:')
print(f'  Total employees: {df_clean["total_employees"].sum():,.0f}')
print(f'  Disabled employees: {df_clean["disabled_employees"].sum():,.0f}')
print(f'\nDisabled Employment Rates:')
print(f'  Average: {df_clean["disabled_percent"].mean():.2f}%')
print(f'  Median: {df_clean["disabled_percent"].median():.2f}%')
print(f'  Min: {df_clean["disabled_percent"].min():.2f}%')
print(f'  Max: {df_clean["disabled_percent"].max():.2f}%')
print(f'\nCompliance Analysis:')
print(f'  Companies with 0 disabled employees: {len(df_clean[df_clean["disabled_employees"] == 0])} ({len(df_clean[df_clean["disabled_employees"] == 0])/len(df_clean)*100:.1f}%)')
print(f'  Companies meeting 4% mandate: {len(df_clean[df_clean["disabled_percent"] >= 4])} ({len(df_clean[df_clean["disabled_percent"] >= 4])/len(df_clean)*100:.1f}%)')
print(f'  Companies below 1%: {len(df_clean[df_clean["disabled_percent"] < 1])} ({len(df_clean[df_clean["disabled_percent"] < 1])/len(df_clean)*100:.1f}%)')

print('\n📋 Sample Data:')
df_results.head(10)

📊 SUMMARY STATISTICS
Total companies processed: 1227
Companies with valid data: 1227

Aggregate Employment:
  Total employees: 8,306,661
  Disabled employees: 43,135

Disabled Employment Rates:
  Average: 0.27%
  Median: 0.04%
  Min: 0.00%
  Max: 6.47%

Compliance Analysis:
  Companies with 0 disabled employees: 548 (44.7%)
  Companies meeting 4% mandate: 3 (0.2%)
  Companies below 1%: 1133 (92.3%)

📋 Sample Data:


,company,total_employees,disabled_employees,disabled_percent,year,xml_url
0,RAJESH EXPORTS LIMITED,76,4,5.263158,2025,https://nsearchives.nseindia.com/corporate/xbr...
1,BF UTILITIES LIMITED,7,0,0.000000,2025,https://nsearchives.nseindia.com/corporate/xbr...
2,Network People Services Technologies Limited,324,1,0.308642,2025,https://nsearchives.nseindia.com/corporate/xbr...
3,KIOCL Limited,249,7,2.811245,2025,https://nsearchives.nseindia.com/corporate/xbr...
4,Sirca Paints India Limited,483,0,0.000000,2025,https://nsearchives.nseindia.com/corporate/xbr...
5,Indraprastha Gas Limited,720,4,0.555556,2025,https://nsearchives.nseindia.com/corporate/xbr...
6,HMT LIMITED,29,0,0.000000,2025,https://nsearchives.nseindia.com/corporate/xbr...
7,THE NEW INDIA ASSURANCE COMPANY LIMITED,10951,321,2.931239,2025,https://nsearchives.nseindia.com/corporate/xbr...
8,Accelya Solutions India Limited,1351,4,0.296077,2025,https://nsearchives.nseindia.com/corporate/xbr...
9,THE ANDHRA SUGARS LIMITED,651,3,0.460829,2025,https://nsearchives.nseindia.com/corporate/xbr...


## Step 7: Top & Bottom Performers

In [8]:
df_clean = df_results[df_results['disabled_percent'].notna()].copy()
df_clean = df_clean.sort_values('disabled_percent', ascending=False)

print('🏆 TOP 10 COMPANIES (Highest disabled employment %)')
print('=' * 70)
print(df_clean[['company', 'total_employees', 'disabled_employees', 'disabled_percent']].head(10).to_string(index=False))

print('\n\n❌ BOTTOM 10 COMPANIES (Excluding zeros)')
print('=' * 70)
df_nonzero = df_clean[df_clean['disabled_employees'] > 0]
print(df_nonzero[['company', 'total_employees', 'disabled_employees', 'disabled_percent']].tail(10).to_string(index=False))

print('\n\n⚠️  ZERO DISABLED EMPLOYEES')
print('=' * 70)
df_zero = df_clean[df_clean['disabled_employees'] == 0]
print(f'Total companies reporting ZERO: {len(df_zero)}')
print('\nExamples:')
print(df_zero[['company', 'total_employees']].head(10).to_string(index=False))

🏆 TOP 10 COMPANIES (Highest disabled employment %)
                          company  total_employees  disabled_employees  disabled_percent
        Lemon Tree Hotels Limited             3677                 238          6.472668
           RAJESH EXPORTS LIMITED               76                   4          5.263158
                IDBI Bank Limited            19087                 874          4.579033
           Uniparts India Limited              484                  19          3.925620
              UNION BANK OF INDIA            73964                2640          3.569304
                     NHPC Limited             3360                 119          3.541667
         VISHAL MEGA MART LIMITED            18029                 629          3.488824
Mazagon Dock Shipbuilders Limited              975                  31          3.179487
                     MMTC LIMITED              287                   9          3.135889
      VENUS PIPES & TUBES LIMITED              385         

## Step 8: Download Results (Google Colab)

In [9]:
# Download final CSV (if running in Colab)
from google.colab import files
files.download('brsr_results/all_companies_disability_data.csv')

print('✅ CSV downloaded! Upload this to your GitHub for visualization.')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ CSV downloaded! Upload this to your GitHub for visualization.
